## Third Attempt on Language Detection

This code shows the third attempt on processing the corpora and trying to come up with a model for the europar.test file.

This code uses files in the /pickles directory. This contains the serialized pickle files with processed tokenized documents, all words and all letters in each one of the documents.

To build a model, this uses the most common words and most common letters used on each language and builds a feature set around them.

In [2]:
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import os
import nltk
import random
import time
import string
import pickle
import numpy

In [3]:
# Utility functions
# These functions were taken from attempt 1 and put together for
# easier maintenance and testing.

def print_elapsed_time():
    end = time.time()
    elapsed = end - start
    m, s = divmod(elapsed, 60)
    h, m = divmod(m, 60)
    return ("%d:%02d:%02d" % (h,m,s))
    
#Remove any <tags> within text
def extract_text_only(text):
    soup = BeautifulSoup(text,"lxml")
#    soup = BeautifulSoup(text,"html5lib")
    return soup.get_text()    

def tokenize_removepuncuation(text):
    #words only 
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(text)

def most_common_words_letter(most_common_words,most_common_letters):
    #Create word_features, a list of most common words on all languauges
    #this will be used on feature set fed to classifier
    word_features = set()
    letter_features = set()

    for k,v in most_common_words.items():
        for word in v:
            word_features.add(word[0])
  
    # Create letter_features, a list of most common letters on all languages
    for k,v in most_common_letters.items():
        for letter in v:
            letter_features.add(letter[0])

    return word_features, letter_features

#Takes two Counter objects, removes common elements
def remove_common_elements(counter1, counter2):
    elements_intersect = (counter1 & counter2).most_common()
    for n in elements_intersect:
        key = n[0]
        del counter1[key]
        del counter2[key]
    return counter1, counter2
    

In [4]:
#Feature extraction and creation functions

#Loop through directory and extract text
#directory is the path to directory to process
def get_text_from_directory(directory):
    language_label = directory.split("/")[-1]    
    documents = []
    counter = 0
    #keep a count on unique words seen on documents
    word_counter = Counter()
    alphabet_counter = Counter()
    alphabet = set()
    #Collect a minimun of 5,000 words per directory(language)
    for filename in os.listdir(directory):
        try:
            text_file = open(directory+"/"+filename,"r").read()
            text = extract_text_only(text_file)
            #Tokenize words and remove punctuation
            tokenized_text = tokenize_removepuncuation(text)
            #add to dict counter
            word_counter.update(tokenized_text)
            #get letters and add to alphabet
            [alphabet_counter.update(list(n)) for n in tokenized_text]
            documents.append((tokenized_text,language_label))
            counter = counter + 1
        except:
            print(directory+" - Issue with filename:"+filename+" Ignoring.")
    return documents, word_counter, alphabet_counter

def extract_data_from_corpora(corpora_directory, number_of_words, number_of_letter, save_pickles):

    all_documents = []
    most_common_words = {}
    most_common_letters = {}

    #Loop through all directories contain corpora with all languages
    #directory will be the folder containing documents on that language
    for directory in os.listdir(corpora_directory):
        #full_path contains
        full_path = corpora_directory+directory
        if(os.path.isdir(full_path)):
            print("About to process directory "+directory)
            #process directory, text contains documents list with rows (['worda1','worda2,'worda3'],'LANG-A')
            #word_counter contains count of all words seen
            text, word_counter, alphabet = get_text_from_directory(full_path)
            print("Number of words for this language:"+str(len(word_counter)))
            #Keep only letters that are not common ascii letters
            for letter in list(alphabet):
                if(letter in list(string.ascii_letters) or letter in list(string.digits)):
                    del alphabet[letter]
            #Keep track of most common words per language to use on feature set
            most_common_words[directory] = word_counter.most_common(number_of_words)
            most_common_letters[directory] = alphabet.most_common(number_of_letter)
            if(save_pickles):
                #Save to pickle so it can be read without having to process again
                pickle_out = open("pickles/word_counter_"+directory+".pickle","wb")
                pickle.dump(word_counter, pickle_out)
                pickle_out.close()
                pickle_out = open("pickles/alphabet_"+directory+".pickle","wb")
                pickle.dump(alphabet, pickle_out)
                pickle_out.close()
                pickle_out = open("pickles/documents_"+directory+".pickle","wb")
                pickle.dump(text, pickle_out)
                pickle_out.close()

            all_documents = all_documents + text
        
    if(save_pickles):
        #Save all_documents to pickle for later use
        pickle_out = open("pickles/all_documents.pickle","wb")
        pickle.dump(all_documents, pickle_out)
        pickle_out.close()
    return all_documents, most_common_words, most_common_letters

#this function receives a document and creates a feature based list
# Input:
#   [(['worda1','worda2,'worda3'],'LANG-A'),
#     ['wordb1','wordb2,'wordb3'],'LANG-B'),...]
# Ouput:
# [('αποφασιστικής': False,
#   'Περιφερειακής': True,
#   'pontot': False,...),'ru'),...]

def document_features(document):
    document_words = set(document)
    features = {}
    #Add word that are part of common words
    for word in word_features:
        features[word] = (word in document_words)
    #Add letters that are part of common letters
    #get letters and add to alphabet
    document_alphabet = set()
    [document_alphabet.update(list(n)) for n in document_words]
    #document_alphabet now contains all letters on this document
    #Keep only letters that are not common ascii letters
    for letter in list(document_alphabet):
        if(letter in list(string.ascii_letters) or letter in list(string.digits)):               
                document_alphabet.remove(letter)
    #Add to feature if exist
    for l in letter_features:
        features[l] = (l in document_alphabet)
    return features


def extract_data_from_corpora_pickles(pickle_directory, number_of_documents, number_of_words, number_of_letters):
    all_documents = []
    most_common_words = {}
    most_common_letters = {}
    
    #Read data from pickle files
    for filename in os.listdir(pickle_directory):
        language = (filename.split('_')[-1]).split('.')[0]
        if('word_counter' in filename):
            all_words_for_language = pickle.load( open( pickle_directory+"/"+filename, "rb" ) )
            ##eliminate any words that intersect with another language
            for lang in most_common_words:
                all_words_for_language, most_common_words[lang] = remove_common_elements(all_words_for_language, most_common_words[lang])

            most_common_words[language] = all_words_for_language
            
        elif('alphabet' in filename):
            all_letters_for_language = pickle.load( open( pickle_directory+"/"+filename, "rb" ) )
            ##eliminate any words that intersect with another language
            for lang in most_common_letters:
                all_letters_for_language, most_common_letters[lang] = remove_common_elements(all_letters_for_language, most_common_letters[lang])
                
            most_common_letters[language] = all_letters_for_language          
        elif('documents' in filename):
            documents = pickle.load( open( pickle_directory+"/"+filename, "rb" ) )
            doc_limit = number_of_documents if(number_of_documents>=0) else len(all_documents) 
            all_documents += documents[:doc_limit]
        
    #return only up to the amount required
    for lang in most_common_words:
         word_limit   = number_of_words if(number_of_words>=0) else len(most_common_words[lang]) 
         most_common_words[lang] = most_common_words[lang].most_common(word_limit)

    for lang in most_common_letters:
         letter_limit   = number_of_letters if(number_of_letters>=0) else len(most_common_letters[lang]) 
         most_common_letters[lang] = most_common_letters[lang].most_common(letter_limit)
    
    return all_documents, most_common_words, most_common_letters

def extract_documents_from_corpora_pickles(pickle_directory, number_of_documents):
    all_documents = []
    
    #Read data from pickle files
    for filename in os.listdir(pickle_directory):
        language = (filename.split('_')[-1]).split('.')[0]
        if('documents' in filename):
            documents = pickle.load( open( pickle_directory+"/"+filename, "rb" ) )
            doc_limit = number_of_documents if(number_of_documents>=0) else len(all_documents) 
            all_documents += documents[:doc_limit]
    
    return all_documents


def extract_wordsletters_from_corpora_pickles(pickle_directory, number_of_words, number_of_letters):
    most_common_words = {}
    most_common_letters = {}
    
    #Read data from pickle files
    for filename in os.listdir(pickle_directory):
        language = (filename.split('_')[-1]).split('.')[0]
        if('word_counter' in filename):
            all_words_for_language = pickle.load( open( pickle_directory+"/"+filename, "rb" ) )
            ##eliminate any words that intersect with another language
            for lang in most_common_words:
                all_words_for_language, most_common_words[lang] = remove_common_elements(all_words_for_language, most_common_words[lang])

            most_common_words[language] = all_words_for_language
            
        elif('alphabet' in filename):
            all_letters_for_language = pickle.load( open( pickle_directory+"/"+filename, "rb" ) )
            ##eliminate any words that intersect with another language
            for lang in most_common_letters:
                all_letters_for_language, most_common_letters[lang] = remove_common_elements(all_letters_for_language, most_common_letters[lang])
                
            most_common_letters[language] = all_letters_for_language          
        
    #return only up to the amount required
    for lang in most_common_words:
         word_limit   = number_of_words if(number_of_words>=0) else len(most_common_words[lang]) 
         most_common_words[lang] = most_common_words[lang].most_common(word_limit)

    for lang in most_common_letters:
         letter_limit   = number_of_letters if(number_of_letters>=0) else len(most_common_letters[lang]) 
         most_common_letters[lang] = most_common_letters[lang].most_common(letter_limit)
    
    return most_common_words, most_common_letters


In [5]:
#Classification functions

def classify_document(document, classifier):
    return classifier.classify(document_features(tokenize_removepuncuation(document)))

def test_europarltest_file(eurofile, resultsfile, everyother, classifier):
    #Read test file and classify each sentence in file
    positive_ctr = 0
    negative_ctr = 0
    total_ctr    = 0 
    #save results to file for processing
    fileout = open(resultsfile,'w')
    #columns
    fileout.write('predicted, language given, correctly classified?\n')
    
    processed_counter = 0
    with open(eurofile,'r') as f:
        for line in f:
            processed_counter +=1
            if(processed_counter%everyother==0):
                total_ctr += 1
                #language is first two letters in line    
                language = line[:2]
                #sentence is rest, clean up spaces
                sentence = line[2:].strip()
                #Detect language based on model
                language_detected = classify_document(sentence, classifier)
                correctly_classified = language_detected==language
                #tally correct and incorrect
                if(correctly_classified):
                    #correctly classified
                    positive_ctr += 1
                else:
                    #incorrectly classified
                    negative_ctr += 1
            
                fileout.write(language_detected+','+language+','+str(correctly_classified)+'\n')
    fileout.close()
    return total_ctr, positive_ctr, negative_ctr

In [6]:
# -------------Step 1-------------
# ----READ FROM PICKLE FILES (Pre-processed files)----
#Get data to create features from corpora
start = time.time()
pickles_directory = "pickles"
#How many documents to process for each language
number_of_documents = 1000
#how many number of most common words per language
number_of_words     = 50 
#letters to use per language
number_of_letters   = 20

all_documents, most_common_words, most_common_letters = extract_data_from_corpora_pickles(pickles_directory, number_of_documents, 100, 100)
print("Elapsed time featureset creation:"+print_elapsed_time())
print("all_documents:"+str(len(all_documents)))
print("most_common_words:"+str(len(most_common_words)))
print("most_common_letters:"+str(len(most_common_letters)))

Elapsed time featureset creation:0:03:39
all_documents:21000
most_common_words:21
most_common_letters:21


In [8]:
# -------------Step 1-------------
# ----READ FROM PICKLE FILES (Pre-read)----
#Get data to create features from corpora
start = time.time()
pickles_directory = "pickles"
number_of_documents = 3000

#Part 1 - get documents
start = time.time()
all_documents = extract_documents_from_corpora_pickles(pickles_directory,number_of_documents) 
print("Elapsed time reading all documents:"+print_elapsed_time())

Elapsed time reading all documents:0:02:41


In [ ]:
number_of_words     = 700 
number_of_letters   = 20
#Part 2 - get common words, letters
start = time.time()
most_common_words, most_common_letters = extract_wordsletters_from_corpora_pickles(pickles_directory, number_of_words, number_of_letters)
print("Elapsed time reading all words, letters:"+print_elapsed_time())

print("all_documents:"+str(len(all_documents)))
print("most_common_words:"+str(len(most_common_words)))
print("most_common_letters:"+str(len(most_common_letters)))

# -------------Step 2-------------
# Create featureset to be used for training
# this is a list of documents with features and label
start = time.time()

#create word_features
word_features, letter_features = most_common_words_letter(most_common_words,most_common_letters)
print("words_features:"+str(len(word_features)))
print("letter_features:"+str(len(letter_features)))

#create featureset
featuresets = [(document_features(d), c) for (d,c) in all_documents]

print("Elapsed time featureset creation:"+print_elapsed_time())
print("featuresets:"+str(len(featuresets)))
# -------------Step 3-------------
# Split train, test for model classification and scoring
numpy.random.shuffle(featuresets)
#calculate how many items to slice by (95% train, 5% test)
slice_by = int((95 * len(featuresets))/100)
train_set, test_set =  featuresets[:slice_by], featuresets[slice_by:]
print("Train set:"+str(len(train_set)))
print("Test set:"+str(len(test_set)))

# -------------Step 4-------------
# Build the Model
start = time.time()
classifier = nltk.NaiveBayesClassifier.train(train_set)
print("Elapsed time for training:"+print_elapsed_time())
# start = time.time()
# print("Accuracy:"+str(nltk.classify.accuracy(classifier, test_set)))
# print("Elapsed time for accuracy testing:"+print_elapsed_time())

#Save classifier for deployment
#Save to pickle so it can be tested later with 
pickle_out = open("models/classifier_"+str(number_of_documents)+"_"+str(number_of_words)+"_"+str(number_of_letters)+".pickle","wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()
pickle_out = open("models/word_features"+str(number_of_documents)+"_"+str(number_of_words)+"_"+str(number_of_letters)+".pickle","wb")
pickle.dump(word_features, pickle_out)
pickle_out.close()
pickle_out = open("models/letter_features"+str(number_of_documents)+"_"+str(number_of_words)+"_"+str(number_of_letters)+".pickle","wb")
pickle.dump(word_features, pickle_out)
pickle_out.close()
print("Done:"+print_elapsed_time())


Elapsed time reading all words, letters:0:00:32
all_documents:66000
most_common_words:21
most_common_letters:21
words_features:14700
letter_features:107


In [13]:
# -------------Step 5-------------
# Test the Model Manually

#sample french document
fr_document = """
Madame la Présidente, il s' agit d' une question sensible pour notre Parlement, et plus précisément relative au débat sur l' élargissement. Je veux parler d' une déclaration du ministère turc des Affaires étrangères au sujet du rapport de M. Poos sur l' adhésion de Chypre à l' Union européenne. Dans cette déclaration, le ministère turc des Affaires étrangères porte une attaque inique et diffamatoire contre un député qui a présenté son rapport à la commission compétente. La commission a adopté ce rapport avec une opposition de deux voix seulement. Je crois comprendre que le système parlementaire n' a pas encore atteint en Turquie un niveau de développement tel que ses membres puissent saisir le contenu du rapport concerné et la responsabilité, si tant est que l' on puisse utiliser ce terme, qu' assume M. Poos. Il s' agit d' un rapport du Parlement. En conséquence, je souhaiterais que le Parlement lui-même apporte une réaction à cette attaque injuste.

Madame la Présidente, permettez-moi de citer un seul exemple de mauvais goût en lisant le point 1 de la déclaration turque :
"le député est connu, d'une part, pour son opposition à la Turquie et, d'autre part, pour ses liens privés avec l'administration chypriote."
C' est une pure calomnie, à la limite d' un délit. Je voudrais que le Parlement, ainsi que la Commission, et non M. Poos, adoptent une réaction à cette accusation lancée par la Turquie !
Je vous remercie, Madame la Présidente !
(Applaudissements)

Je vous remercie. Nous allons regarder cela de très près.
Il n'y a pas d'autre motion d'ordre.
Je dirai simplement à M. Helmer que les fonctionnaires sont actuellement occupés à remettre le drapeau britannique à sa place ; il y avait, en effet, ce matin un problème de drapeau britannique que nous avons tenu à résoudre sans attendre.

Situation au Moyen-Orient
L'ordre du jour appelle la déclaration du Conseil et de la Commission sur la situation au Moyen-Orient.

Madame la Présidente, Monsieur le Commissaire, Mesdames et Messieurs, tout d'abord veuillez excuser mon petit retard, mais je viens en droite ligne de Bruxelles. J'espère que vous voudrez bien le comprendre et je vous en remercie.
Mesdames et Messieurs, comme le ministre Louis Michel l'avait déjà indiqué lors de sa comparution, le 28 août dernier, devant la commission des affaires étrangères, des droits de l'homme, de la sécurité commune et de la politique de défense de ce Parlement, on assiste depuis des mois au Proche-Orient à une escalade croissante de la violence, avec pour résultat consternant l'effondrement total de la confiance mutuelle entre les parties et, sur le terrain, cela a créé un profond sentiment d'impuissance parmi toutes les populations concernées.
Il ne se passe hélas pas un jour ou à peu près sans que des incidents sanglants et de nouvelles provocations ne se produisent et ne reportent ainsi indéfiniment la matérialisation d'un cessez-le-feu et la fin d'un cycle infernal de représailles, tandis que la liste des victimes ne fait que s'allonger. Cette situation, récemment qualifiée par le ministre Vedrine de monstrueuse et révoltante, suscite bien entendu une profonde inquiétude pour la stabilité d'une région qui est à nos portes. La communauté internationale ne saurait tolérer plus longtemps cette escalade et se doit de condamner avec fermeté les facteurs d'aggravation que constituent notamment le terrorisme et les attentats suicide perpétrés par des Palestiniens en Israël. Outre que ces actes terroristes constituent une abomination, car ils frappent des civils innocents, ils ne font qu'inciter Israël à pratiquer une politique de plus en plus répressive.
Les tirs d'activistes palestiniens contre des Israéliens, qu'ils soient colons ou militaires, à partir d'agglomérations sous contrôle palestinien, mais aussi le recours excessif et disproportionné à la force par Israël ne font qu'alimenter le cycle de la violence. L'usage d'avions de combat dans les zones résidentielles, la destruction systématique par des missiles de bâtiments abritant les services de police et de sécurité de l'autorité palestinienne et les meurtres ciblés d'activistes palestiniens ne constituent pas des éléments convaincants d'une stratégie visant à la paix et à la sécurité. Les incursions militaires israéliennes dans les zones passées sous contrôle palestinien sont autant de violations des accords signés. La fermeture des institutions palestiniennes à Jérusalem-Est, et notamment celle de la Maison Orient, et la saisie des archives sont des mesures politiques peu propices à l'apaisement.
Madame la Présidente, Mesdames et Messieurs, lorsque nous observons aujourd' hui les perspectives d' une reprise du dialogue direct, où en sommes-nous ? Pour commencer, c' est dans un contexte très inquiétant de radicalisation des différentes parties impliquées dans le conflit que le ministre israélien des Affaires étrangères, M. Shimon Peres, a récemment proposé, en public, la reprise d' un dialogue direct avec l' Autorité palestinienne en vue de réduire la violence et de veiller à ce que le cessez-le-feu soit respecté, deux objectifs qui semblaient inaccessibles jusqu' ici. Au cours d' une récente visite dans la région, le ministre allemand des Affaires étrangères, M. Joschka Fischer, a obtenu que le dirigeant palestinien accepte une telle rencontre entre M. Arafat, donc, d' un côté et M. Shimon Peres de l' autre.
"""

#sample spanish document
es_document = """
Nombramiento del Presidente del Banco Central Europeo
 De conformidad con el orden del día, se procede al debate de la recomendación (A5-0307/2003), en nombre de la Comisión de Asuntos Económicos y Monetarios, relativa al nombramiento del Sr. Jean-Claude Trichet como Presidente del Banco Central Europeo (10893/2003 - C5-0332/2003 - 2003/0819(CNS)) (Ponente: Sra. Randzio-Plath)
. (IT) Señor Presidente, Señorías, señores Comisarios, es un gran placer hablar sobre una cuestión de gran importancia para la Unión Europea: el nombramiento del Presidente del Banco Central Europeo.
La creación del euro es un éxito considerable en la historia de la integración europea, tanto desde el punto de vista político como técnico. El euro sigue siendo una divisa relativamente nueva, y tendrá que basarse en la experiencia, ampliamente reconocida, del Banco Central Europeo para continuar siendo un éxito.
En este momento estamos sustituyendo por primera vez al Presidente del Banco Central Europeo. Es de vital importancia para el Banco Central Europeo y para el euro que la elección de un sucesor del Presidente Duisenberg se lleve a cabo con transparencia y que se base únicamente en los criterios recogidos en el Tratado que establece la Comunidad Europea, así como en el Estatuto del Sistema Europeo de Bancos Centrales y el del Banco Central Europeo, y en el acuerdo de este último sobre el candidato más cualificado para el trabajo.
Al nombrar a un sucesor, hemos de reconocer y rendir tributo al excelente trabajo realizado por el Presidente Duisenberg, pero también debemos expresar nuestra confianza en que el Banco Central Europeo seguirá desarrollando todas sus funciones con el mismo grado de éxito que hasta la fecha.
La base legal para el procedimiento de nombramiento de un nuevo Presidente del Banco Central Europeo se encuentra en la letra b del apartado 2 del artículo 112, y en apartado 4 del artículo 122, del Tratado que establece la Comunidad Europea, así como en los artículos 11.2 y 43.3 del Protocolo del Estatuto del Sistema Europeo de Bancos Centrales y del Banco Central Europeo.
De acuerdo con estas disposiciones, el Consejo de «Información» Ecofin adoptó una recomendación el 15 de julio de 2003 que defendía el nombramiento del Sr. Trichet como Presidente del Banco Central Europeo por un periodo de ocho años con efecto a partir del 1 de noviembre de 2003. La recomendación fue enviada tanto a ustedes como al Banco Central Europeo, de acuerdo con el Tratado, para que pudieran dar su opinión antes de que la recomendación fuera presentada a los Jefes de Estado o de Gobierno para la decisión final.
El Consejo de Gobierno del Banco Central Europeo aprobó su opinión el 31 de julio y la envió al Consejo y al Parlamento. Esta opinión confirmó que el Consejo Regulador del Banco Central Europeo cree que el candidato propuesto deber ser una persona de reconocido prestigio y experiencia profesional en materia monetaria y bancaria, como establece la letra b del apartado 2 del artículo 112 del Tratado.
Espero que el Parlamento Europeo esté de acuerdo con el Consejo y con el Banco Central Europeo en que el Sr. Trichet es un candidato excelente para este importante puesto. La adopción de la opinión por parte del Parlamento permitirá que los Jefes de Estado o de Gobierno tomen una decisión final sobre la toma de posesión del nuevo Presidente del Banco Central Europeo, dentro del calendario fijado por la recomendación del Consejo.

Señor Presidente, en nombre de la Comisión de Asuntos Económicos y Monetarios, recomiendo que el Parlamento confirme el nombramiento del Sr. Jean-Claude Trichet como candidato adecuado para el puesto de Presidente del Banco Central Europeo. El candidato nominado ha presentado una declaración por escrito y ha dado explicaciones orales a dicha comisión en el curso del proceso de confirmación. Ha convencido a los miembros de dicha comisión no solo de su integridad personal y competencia profesional, sino también de sus visión de la política económica y monetaria en la Eurozona. Al mismo tiempo, ha demostrado ser receptivo a las exigencias de una mayor transparencia y responsabilidad democrática en el seno del Banco Central Europeo.
Cinco años después de su fundación, el Banco Central Europeo ya es mayor de edad. Su independencia -en términos políticos, económicos, financieros, organizativos y de personal- está garantizada y no está cuestionada por el Tratado que establece la Constitución para Europa. Su alto grado de independencia, que supera al de la Reserva Federal de los Estados Unidos, significa que el BCE tiene una gran responsabilidad en el desarrollo macroeconómico y social. Ello requiere la mayor transparencia posible en interés de la democracia y de la política de integración. Por ello, la transparencia de las decisiones y del procedimiento de toma de decisiones es parte integral del papel del Banco Central Europeo. Este esfuerzo para lograr la transparencia se refleja en el diálogo monetario trimestral con el Parlamento Europeo, en las publicaciones y decisiones ordinarias, pero también en los informes, las conferencias y las previsiones sobre la inflación que se publican cada seis meses. Así pues, en Europa ha tenido lugar una especie de revolución cultural. La cultura de los bancos centrales nacionales en Europa no contaba con este tipo de transparencia. Por cierto, la transparencia también va en interés del BCE porque es todavía una institución nueva, y por lo tanto, se basa especialmente en el establecimiento y la consolidación de su legitimidad, la credibilidad y confianza como autoridad europea.
En última instancia, el sistema monetario de una nación refleja todo lo que defiende esa nación y todo a lo que aspira y mantiene, si se me permite citar al renombrado economista europeo Josef Schumpeter. Creo que en la fase actual del debate, la incorporación del Tratado de Maastricht en su totalidad en el borrador del Tratado que establece la Constitución para Europa era el paso más adecuado. A lo largo de los siglos de su existencia, el papel de los bancos centrales ha sufrido un cambio radical, comenzando por su forma de organización privada en la historia, pasando por su cambio de estatus en los Estados Unidos, el concepto de lucha contra la inflación, hasta llegar a su papel de banco emisor independiente. Quizás sea demasiado pronto para encontrar respuesta a los retos actuales. La tarea que debemos emprender es definir el papel del Banco Central en un mundo globalizado dominado por el comercio y los mercados financieros internacionales. Ello implica mercados dinámicos, pero también riesgos cada vez mayores para la estabilidad financiera internacional.
¿Qué papel pueden y deben desempeñar, pues, los bancos centrales para contribuir a la estabilidad financiera, evitar las crisis financieras y proporcionar asistencia? ¿Está preparado el BCE para el papel de prestamista en última instancia? ¿Es esto lo que queremos? El euro también ha mejorado la posición internacional de Europa. El BCE tendrá que desempeñar un papel cada vez mayor en la definición y aplicación de las políticas apropiadas para una economía globalizada. Estamos preocupados por el gran desequilibrio de la economía estadounidense y los riesgos que pudiera entrañar para cualquier otra parte del mundo a medio plazo.
"""

print("Classification of Spanish document:"+classify_document(es_document,classifier))
print("Classification of French document:"+classify_document(fr_document,classifier))

Classification of Spanish document:sk
Classification of French document:sk


In [ ]:
# -------------Step 6-------------
# Classify all sentences in europarl.test and write results to resultsfile
# This is the actual deployment of the classifier against challenge data
europarl_testfile = "europarl.test"
results_outfile   = "europarl_test_classified_attempt4.csv"
#use for quick testing, to test just a subset of all documents read
#every other 1000 would only classify every 1000 document on testfile
everyother = 500
start = time.time()
total_ctr, positive_ctr, negative_ctr = test_europarltest_file(europarl_testfile, results_outfile, everyother, classifier)
#results
print("       Total attempted: "+str(total_ctr))
print("  Classified correctly: "+str(positive_ctr))
print("Classified incorrectly: "+str(negative_ctr))
accuracy = (positive_ctr/total_ctr) * 100
print("              Accuracy: %s",accuracy)
print("Elapsed time for accuracy testing:"+print_elapsed_time())